In [18]:
import os
from docx import Document
import pandas as pd
import re
# openai предоставляет только библиотеку для формирования запросов к языковым моделям
# ChatGPT не используется
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import Dict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


def load_documents_from_folder(path, prefix):
    # Собираем список файлов в папке
    files = [f for f in os.listdir(path) if f.endswith('.docx') and f.startswith(prefix)]
    data = []
    for file in files:
        # Извлекаем номер из названия файла
        match = re.search(rf'{prefix}(\d+)', file)
        if match:
            file_number = match.group(1)
        else:
            continue  # Пропускаем файлы без номера

        # Полный путь к файлу
        filepath = os.path.join(path, file)
        doc = Document(filepath)
        paragraphs = [p.text for p in doc.paragraphs]  # Извлекаем текст из всех абзацев
        if paragraphs:
            first_line = paragraphs[0]
            name_match = re.search(r'\](.*)', first_line)
            if name_match:
                document_name = name_match.group(1).strip()  # Извлекаем название документа
            else:
                document_name = "No Name Found"  # Заглушка, если название не найдено
        else:
            document_name = "Empty Document"  # Заглушка для пустых документов

        full_text = '\n'.join(paragraphs)  # Объединяем текст в одну строку
        data.append({'Number': file_number, 'Name': document_name, 'Text': full_text})
    return pd.DataFrame(data)

# Пути к папкам
path_hmi = './HMI'
path_stss = './SSTS'

# Загружаем документы из обеих папок
df_hmi = load_documents_from_folder(path_hmi, 'UC-')
df_stss = load_documents_from_folder(path_stss, 'SSTS-')

# Объединяем датафреймы по номеру документа
df = pd.merge(df_hmi, df_stss, on='Number', how='left', suffixes=('_HMI', '_SSTS'))

# Дропаем колонку Name_SSTS
df = df.drop(columns=['Name_SSTS'])

# Переименовываем колонку Name_HMI в Name
df = df.rename(columns={'Name_HMI': 'Name'})

print(df)


    Number                                               Name  \
0    30365         Start the Charging Process via Soft Switch   
1   259572                  Mute/unmute the FM Radio playback   
2    29448                        Configure heat preservation   
3    65833                 Driver initiate a call through SWP   
4   261611                        FM Radio Stations switching   
5   315231            Adding Internet Radio to Favorites List   
6   114671  Revoke access to the vehicle from a driver or ...   
7    30364      Stop charging when the cut-off SOC is reached   
8    65832                 Driver initiate a call through SWP   
9    86921      Emergency Service Communication (ERA-Glonass)   
10   30370             Stop the discharging process (Updated)   
11   88001                       Receiving Call Notifications   
12   88002                       Receiving Call Notifications   
13  259571                  Mute/unmute the FM Radio playback   
14   65831               

In [19]:
from typing import List, Optional
import pandas as pd
import requests
from pydantic import BaseModel

class Message(BaseModel):
    role: str
    content: str
    
# Payload example
class Payload(BaseModel):
    model: Optional[str] = None
    messages: List[Message]

# Функция `sent_to_ai` предназначена для отправки запроса в ЛОКАЛЬНО развернутую llm модель
# Данные никуда не отправляются и остаются в безопасности на вашем устройстве
# Она принимает данные в формате словаря (payload), которые содержат параметры для выполнения запроса к модели
# В результате выполнения запроса функция получает ответ от модели и возвращает его текстовое содержание

# DEFAULT_MODEL = 'llama-3.2-3b-instruct'
DEFAULT_MODEL = 'meta-llama-3.1-8b-instruct'
    
def sent_to_ai(payload: Payload):
    if not payload.model:
        payload.model = DEFAULT_MODEL
        
    payload = payload.dict()
    
    try:
        client = OpenAI(base_url="http://127.0.0.1:1234/v1/", api_key="lm-studio")
        response = client.chat.completions.create(**payload)
        print('response', response)
        
        return response.choices[0].message.content
    except Exception as e:
        # Обработка любой ошибки
        print(f"Произошла ошибка: {e}")




In [20]:
def create_short_hmi_description(text_hmi):
    prompt = f"""
    Analyze the following detailed description of an HMI (Human-Machine Interface) and summarize it in a concise and clear format. Focus on the main functions, user interactions, and essential controls or indicators. The summary should be brief, straightforward, and formatted in a way that makes it easy for further verification or testing.

    Original HMI Description: "{text_hmi}"

    Please provide a short summary focusing on the key functionalities and interactions in the HMI. Avoid any additional explanations, and keep the language simple and direct.
    """
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)
    response = sent_to_ai(payload)
    return response.strip()  # Возвращаем краткое описание без лишних символов

# Применение функции для создания новой колонки 'Short HMI'
df['Short HMI'] = df['Text_HMI'].apply(create_short_hmi_description)

response ChatCompletion(id='chatcmpl-l7yt3ilc1fgzs4frctfkem', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**HMI Summary: Start Charging Process via Soft Switch**\n\n**Main Functions:**\n\n1. Navigate to charging interface\n2. Check vehicle status (Parking mode)\n3. Check charging cable connection\n4. Start charging process\n5. Display notification on successful start\n\n**User Interactions:**\n\n1. Press "Start Charging" soft switch button\n\n**Essential Controls/Indicators:**\n\n1. Soft switch button ("Start Charging")\n2. Notification display (successful start)\n\n**Preconditions:**\n\n1. Vehicle is in Parking mode\n2. Charging cable is plugged in', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1731223114, model='meta-llama-3.1-8b-instruct', object='chat.completion', service_tier=None, system_fingerprint='meta-llama-3.1-8b-instruct', usage=CompletionUsage(completion_tokens=117, prom

In [21]:
# Шаблон промпта для фильтрации нерелевантного контента
keep_only_relevant_content_prompt_template = """
You are provided with a query and a set of retrieved documents. Your task is to filter out all non-relevant information that does not provide important details regarding the query. 

Query: {query}

Retrieved Documents: {retrieved_documents}

Your goal is to keep only the information that is directly relevant to the query, removing any extraneous details. You may remove parts of sentences or entire sentences that are not relevant to the query.

DO NOT ADD ANY NEW INFORMATION THAT IS NOT IN THE RETRIEVED DOCUMENTS.

Output only the filtered, relevant content.
"""

# Модель для вывода релевантного контента
class KeepRelevantContent(BaseModel):
    relevant_content: str = Field(description="The relevant content from the retrieved documents that is relevant to the query.")

# Подготовка промпта для запроса
def create_relevant_content_prompt(short_hmi, text_stss):
    return keep_only_relevant_content_prompt_template.format(
        query=short_hmi,
        retrieved_documents=text_stss
    )

# Функция для фильтрации контента
def keep_only_relevant_content(row):
    """
    Filters Text_SSTS to keep only the content relevant to Short HMI.

    Args:
        row: A row of the DataFrame containing 'Short HMI' and 'Text_SSTS'.

    Returns:
        str: The filtered relevant content from 'Text_SSTS' based on 'Short HMI'.
    """
    # Проверяем, есть ли данные в Text_STSS
    if pd.isna(row['Text_SSTS']):
        return ""

    # Формируем промпт для фильтрации контента
    prompt = create_relevant_content_prompt(row['Short HMI'], row['Text_SSTS'])
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)
    response = sent_to_ai(payload)

    # Возвращаем отфильтрованный релевантный контент
    return response.strip()

# Применение функции к DataFrame для обновления колонки 'Text_STSS'
df['Filtered Text_SSTS'] = df.apply(keep_only_relevant_content, axis=1)

response ChatCompletion(id='chatcmpl-1fs9vlgvtggooa1sluz8i', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Relevant Content:**\n\n1. The power mode of the vehicle is CGW_VehicleState=Standby-Normal/ StandbyOnlyLVon/Drive-Normal.\n2. IVI_IFT can receive the status signal of the corresponding controller.\n3. The user can operate the display.\n\n**Trigger Conditions for Starting Charging:**\n\n1. IVI receives vehicle/{VIN}/hardware/battery/chargingState = 0x5: AC Charging/0x6: DC Charging/0xA: AC Preheating/0xB: DC Preheating.\n2. IVI receive vehicle/Powertrain/TractionBattery/Charging/ChargeFormSet = 0x2: Appointment charging & vehicle/{VIN}/hardware/battery/chargingPileState = 0x2: 9V PWM & vehicle/{VIN}/hardware/battery/chargingConnectorState = 0x2: 1500 Ohm.\n3. IVI receive vehicle/{VIN}/hardware/battery/chargingLockState = 0x1: close.\n\n**Execution Output for Starting Charging:**\n\n1. IVI_IFT displays the "Start Charging" swi

In [22]:
def answer_question_from_context(row):
    """
    Answers a question about whether 'Short HMI' meets the requirements in 'Text_SSTS' using a chain-of-thought reasoning approach.

    Args:
        row: A row of the DataFrame containing 'Short HMI' and 'Text_SSTS'.

    Returns:
        str: The answer based on the reasoning chain.
    """
    # Формируем контекст и вопрос для цепочки рассуждений
    context = f"""
    Short HMI: {row['Short HMI']}
    Requirements (Text_SSTS): {row['Text_SSTS']}

    """
    
    question = "Does the 'Short HMI' meet the requirements outlined in 'Text_SSTS'?"

    # Промпт с примерами цепочки рассуждений
    prompt = f"""
    Examples of Chain-of-Thought Reasoning

    Example 1
    Context:
    Short HMI: "The vehicle's dashboard displays the current speed in kilometers per hour."
    Requirements (Text_SSTS): "The vehicle must display the speed in both kilometers per hour and miles per hour."
    Question: Does the 'Short HMI' meet the requirements outlined in 'Text_SSTS'?
    Reasoning Chain:
    The Short HMI states that the dashboard displays the current speed in kilometers per hour.
    The requirement specifies that the speed must be displayed in both kilometers per hour and miles per hour.
    Since the Short HMI only mentions kilometers per hour, it does not include miles per hour.
    Therefore, the 'Short HMI' does not fully meet the requirements outlined in 'Text_SSTS'.

    Example 2
    Context:
    Short HMI: "The vehicle is equipped with a touchscreen interface that allows users to control the air conditioning system, including temperature settings, fan speed, and airflow direction."
    Requirements (Text_SSTS): "The human-machine interface must provide controls for the air conditioning system, including temperature adjustment, fan speed control, and airflow direction selection."
    Question: Does the 'Short HMI' meet the requirements outlined in 'Text_SSTS'?
    Reasoning Chain:
    The Short HMI describes a touchscreen interface that allows users to control temperature settings, fan speed, and airflow direction.
    The requirements specify that the HMI must provide controls for temperature adjustment, fan speed control, and airflow direction selection.
    The Short HMI includes all the controls specified in the requirements.
    Therefore, the 'Short HMI' meets the requirements outlined in 'Text_SSTS'.

    Now, use the following context and question to provide an answer with step-by-step reasoning:
    Context:
    {context}
    Question:
    {question}
    """

    # Подготовка сообщений и отправка запроса к GPT
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)
    response = sent_to_ai(payload)
    # Возвращаем только ответ
    return response.strip()

# Применение функции к DataFrame для создания колонки с ответами
df['Requirement Compliance Answer'] = df.apply(answer_question_from_context, axis=1)



response ChatCompletion(id='chatcmpl-sl0xneeqsw9q9mnpq7551q', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="To determine if the 'Short HMI' meets the requirements outlined in 'Text_SSTS', we will follow a step-by-step reasoning chain:\n\n1.  The Short HMI allows users to start the charging process via a soft switch button.\n2.  The requirement mentions that users can choose between starting or stopping charging on IVI_IFT, which includes unlocking/locking the charging gun.\n3.  However, the Short HMI only provides a 'Start Charging' soft switch button and does not explicitly mention stopping the charging process or controlling the electronic lock.\n4.  The requirement specifies enabling conditions for IVI_IFT to receive status signals from controllers and trigger conditions for IVI_IFT to display specific switches based on vehicle states.\n5.  Upon examining the Short HMI, we find that it includes preconditions such as the vehicle 

In [23]:


# Загрузка модели FLAN-T5
# model_name = "google/flan-t5-base"  # Вы можете использовать 'google/flan-t5-large' для более точных результатов, если у вас достаточно ресурсов
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


# def check_compliance(short_hmi, text_ssts):
#     # Подготавливаем запрос для FLAN-T5
#     prompt = f"Determine if the following description complies with the technical regulation. If it does not, specify the discrepancies. Answer 'Yes' or 'No' followed by the details. Description: '{short_hmi}' Technical Regulation: '{text_ssts}'"

#     # Генерируем ответ с помощью FLAN-T5
#     inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
#     outputs = model.generate(inputs, max_length=256, num_beams=5, early_stopping=True)
#     answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     print(answer)
#     return answer
# df['Compliance_Check'] = df.apply(lambda row: check_compliance(row['Short HMI'], row['Text_SSTS']), axis=1)

In [24]:
def summarize_discrepancies(description):
    prompt = f"""
    Given a detailed description of compliance discrepancies, summarize the key issues succinctly. Focus only on the main points of non-compliance and essential details that highlight what is wrong or missing. Please provide a concise summary.

    Full Description: "{description}"
    """
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)  # Используйте соответствующую модель GPT
    response = sent_to_ai(payload)
    return response.strip()

# Применение функции к DataFrame для создания новой колонки 'Differences'
df['Differences'] = df['Requirement Compliance Answer'].apply(summarize_discrepancies)


response ChatCompletion(id='chatcmpl-vfcnwdlr33jksr1t2wb47', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here\'s a concise summary of the key issues with the "Short HMI" compliance discrepancies:\n\n1. **Missing "Stop Charging" functionality**: The Short HMI lacks the ability to stop the charging process.\n2. **Inadequate control over electronic lock**: It does not provide switches for unlocking/locking the electronic lock, as required by \'Text_SSTS\'.\n3. **Preconditions mismatch**: The preconditions in the Short HMI do not align with the enabling or trigger conditions specified in the requirements.\n4. **Execution output omissions**: The execution output of the Short HMI does not display switches for critical actions (stopping charging, locking/unlocking electronic lock).', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1731223286, model='meta-llama-3.1-8b-instruct', object='chat.co

In [25]:
def create_description(row):
    prompt = f"""
    Given the detailed descriptions from three sources - HMI interface operations, system standards and specifications (STSS), and key discrepancies - synthesize a clear and concise summary that captures the essence of the functionality or process described. Focus on integrating information to provide a comprehensive yet succinct description that is useful for understanding the core functionalities and any critical issues.

    HMI Description: "{row['Text_HMI']}"
    STSS Description: "{row['Text_SSTS']}"
    Key Discrepancies: "{row['Differences']}"

    Please produce a summary that combines these details into a streamlined description of the main features and functions, highlighting any significant discrepancies where applicable. The summary should be concise, informative, and only include the most relevant information for clarity and utility.
    """
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)
    response = sent_to_ai(payload)
    return response.strip()

# Применение функции к DataFrame
df['Description'] = df.apply(create_description, axis=1)


response ChatCompletion(id='chatcmpl-i7i5jr7putweb9xi9vyo', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Summary of Charging Process Functionality**\n\nThe charging process can be initiated via the Soft Switch on the vehicle\'s interface or mobile app when the vehicle is in Parking mode and the charging cable is plugged in. The system checks these preconditions before starting the charging process.\n\n**Main Features:**\n\n1.  **Starting Charging:** The user navigates to the charging interface, presses the "Start Charging" soft switch button, and receives a notification that the charging session has been successfully started.\n2.  **Immediate Charging:** The user can choose to start charging immediately if the vehicle is in appointment mode and the charging connector state matches specific conditions (9V PWM with a resistance of 1500 Ohm or less).\n3.  **Unlocking Electronic Lock:** If the charging state indicates an AC plug det

In [26]:
# Модель данных для проверки фактической обоснованности ответа
class IsGroundedOnFacts(BaseModel):
    grounded_on_facts: str = Field(description="Answer 'True' if the answer is based on the provided context, 'False' otherwise.")

# Шаблон промпта для проверки фактической обоснованности ответа
is_grounded_on_facts_prompt_template = """
You are provided with a context and an answer. Your task is to determine if the answer is fully grounded in the information provided within the context. 

Context:
{context}

Answer:
{answer}

Please respond with "True" if the answer is completely based on the given context, without any unsupported assumptions or additional information, or "False" if the answer includes any hallucinations or unsupported claims.
"""

# Функция для создания промпта
def create_is_grounded_prompt(context, answer):
    return is_grounded_on_facts_prompt_template.format(
        context=context,
        answer=answer
    )

# Основная функция для проверки обоснованности ответа
def is_answer_grounded_on_context(state):
    """
    Determines if the answer to the question is grounded in the facts.
    
    Args:
        state: A dictionary containing the context and answer.
    
    Returns:
        str: "grounded on context" if the answer is based on the context, "hallucination" otherwise.
    """
    print("Checking if the answer is grounded in the facts...")
    context = state["context"]
    answer = state["answer"]

    # Создание промпта для проверки
    prompt = create_is_grounded_prompt(context, answer)
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)
    response = sent_to_ai(payload)

    # Проверка ответа: True - обоснован, False - содержит галлюцинации
    if response.strip().lower() == "true":
        print("The answer is grounded in the facts.")
        return "grounded on context"
    else:
        print("The answer is hallucination.")
        return "hallucination"


In [27]:
def send_to_gpt_for_rating(row):
    # Проверка на применимость требований
    if pd.isna(row['Text_SSTS']):
        return "NA"  # Возвращаем "NA", если требования не применимы

    prompt = f"""
    Given comprehensive descriptions from multiple sources, analyze the text and classify it into one of the specific compliance categories based solely on the content provided. You should return ONLY the abbreviation of the compliance level without any additional text or explanation.

    Categories:
    - FC (Fully Compliant): The situation is perfect and nothing can be improved.
    - LC (Largely Compliant): Generally correct, but some minor improvements may be needed. No full review is necessary.
    - PC (Partially Compliant): There are major deviations. Significant improvements are needed and a subsequent review is required.
    - NC (Non-Compliant): The requirements are not met, necessitating a complete redo and re-review.
    - NA (Not Applicable): The situation described does not apply to the standards or requirements in question.

    HMI Description: "{row['Text_HMI']}"
    System Standards and Specifications (SSTS): "{row['Text_SSTS']}"
    Key Discrepancies: "{row['Requirement Compliance Answer']}"
    Differences: "{row['Differences']}"
    Integrated Description: "{row['Description']}"

    Based on the above information, please provide only the two-letter abbreviation (FC, LC, PC, NC, NA) that best describes the overall compliance level.
    """
    messages = [Message(role="user", content=prompt)]
    payload = Payload(messages=messages)
    response = sent_to_ai(payload)
    return response.strip()  # Ensure only the abbreviation is returned with no extra spaces or characters

# Применение функции к DataFrame
df['Complience Level'] = df.apply(send_to_gpt_for_rating, axis=1)



response ChatCompletion(id='chatcmpl-5y68w6hoiykl7ordtl524', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='PC', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1731223377, model='meta-llama-3.1-8b-instruct', object='chat.completion', service_tier=None, system_fingerprint='meta-llama-3.1-8b-instruct', usage=CompletionUsage(completion_tokens=1, prompt_tokens=2808, total_tokens=2809, completion_tokens_details=None, prompt_tokens_details=None))
response ChatCompletion(id='chatcmpl-3tjk6jyt22rqw4okzgjn9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='PC', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1731223378, model='meta-llama-3.1-8b-instruct', object='chat.completion', service_tier=None, system_fingerprint='meta-llama-3.1-8b-instruct', usage=CompletionUsage(completion_tokens=1, prompt_token

In [28]:
submission = df[['Number', 'Name', 'Differences', 'Description', 'Complience Level']]
submission.to_csv('submission.csv', index = False)
print(submission)

    Number                                               Name  \
0    30365         Start the Charging Process via Soft Switch   
1   259572                  Mute/unmute the FM Radio playback   
2    29448                        Configure heat preservation   
3    65833                 Driver initiate a call through SWP   
4   261611                        FM Radio Stations switching   
5   315231            Adding Internet Radio to Favorites List   
6   114671  Revoke access to the vehicle from a driver or ...   
7    30364      Stop charging when the cut-off SOC is reached   
8    65832                 Driver initiate a call through SWP   
9    86921      Emergency Service Communication (ERA-Glonass)   
10   30370             Stop the discharging process (Updated)   
11   88001                       Receiving Call Notifications   
12   88002                       Receiving Call Notifications   
13  259571                  Mute/unmute the FM Radio playback   
14   65831               